In [1]:
import pandas as pd
import wmfdata as wmf

You are using wmfdata 0.1.0 (latest).

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


# Missing revision IDs

All EditAttemptStep events that aren't creating a new page (identified by `page_id != 0`) should have a valid `revision_id` (corresponding newly saved revision for `saveSuccess` events, and the base revision for the edit for all the rest).

However, some events don't.

In [12]:
missing_rev_ids = wmf.hive.run("""
SELECT
  event.platform AS platform,
  event.editor_interface AS editor_interface,
  event.action AS action,
  COUNT(*) AS events
FROM event.editattemptstep
WHERE
  event.page_id != 0 AND
  (event.revision_id IS NULL OR event.revision_id = 0) AND
  year > 0
GROUP BY
  event.platform,
  event.editor_interface,
  event.action
""").pivot_table(
  index=["platform", "editor_interface"], 
  values="events",
  columns="action",
  aggfunc="sum"
).fillna(0).applymap(int)

In [32]:
(
  missing_rev_ids
  .applymap(wmf.utils.num_str)
  .fillna(0)
)

action                       abort firstChange     init     loaded      ready  \
platform editor_interface                                                       
desktop  visualeditor       53,000      28,000   71,000     63,000     63,000   
         wikitext          820,000   1,200,000        0  2,400,000  2,400,000   
         wikitext-2017          12          31       26         21         23   
phone    visualeditor          130      36,000  210,000    120,000    120,000   
         wikitext           67,000      21,000  470,000    400,000    400,000   

action                    saveAttempt saveFailure saveIntent saveSuccess  
platform editor_interface                                                 
desktop  visualeditor          22,000       5,300     21,000         220  
         wikitext                   0           0          0           0  
         wikitext-2017              9           1         14           0  
phone    visualeditor          23,000       3,500     27,000         580  
         wikitext              17,000       1,100     23,000         280

In [33]:
_32.reset_index().pipe(wmf.utils.df_to_remarkup)

| platform | editor_interface | abort | firstChange | init | loaded | ready | saveAttempt | saveFailure | saveIntent | saveSuccess
| ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- 
| desktop | visualeditor | 53,000 | 28,000 | 71,000 | 63,000 | 63,000 | 22,000 | 5,300 | 21,000 | 220
| desktop | wikitext | 820,000 | 1,200,000 | 0 | 2,400,000 | 2,400,000 | 0 | 0 | 0 | 0
| desktop | wikitext-2017 | 12 | 31 | 26 | 21 | 23 | 9 | 1 | 14 | 0
| phone | visualeditor | 130 | 36,000 | 210,000 | 120,000 | 120,000 | 23,000 | 3,500 | 27,000 | 580
| phone | wikitext | 67,000 | 21,000 | 470,000 | 400,000 | 400,000 | 17,000 | 1,100 | 23,000 | 280



In [14]:
total_actions = wmf.hive.run("""
SELECT
  event.platform AS platform,
  event.editor_interface AS editor_interface,
  event.action AS action,
  COUNT(*) AS events
FROM event.editattemptstep
WHERE
  year > 0
GROUP BY
  event.platform,
  event.editor_interface,
  event.action
""").pivot_table(
  index=["platform", "editor_interface"], 
  values="events",
  columns="action",
  aggfunc="sum"
).fillna(0).applymap(int)

In [26]:
(
  (missing_rev_ids / total_actions)
  .fillna(0)
  .applymap(wmf.utils.pct_str)
)

action                     abort firstChange  init loaded  ready saveAttempt  \
platform editor_interface                                                      
desktop  visualeditor       3.4%       12.6%  4.3%   4.1%   4.1%       12.0%   
         wikitext          51.4%       88.0%  0.0%  70.1%  70.1%        0.0%   
         wikitext-2017      0.1%        0.1%  0.1%   0.0%   0.1%        0.0%   
phone    visualeditor       0.0%        4.0%  4.3%   3.7%   3.7%        3.5%   
         wikitext           1.5%        3.6%  5.5%   4.9%   4.9%        3.5%   

action                    saveFailure saveIntent saveSuccess  
platform editor_interface                                     
desktop  visualeditor           13.2%      11.9%        0.1%  
         wikitext                0.0%       0.0%        0.0%  
         wikitext-2017           0.1%       0.0%        0.0%  
phone    visualeditor            4.2%       3.7%        0.1%  
         wikitext                3.3%       3.2%        0.1%

In [28]:
_26.reset_index().pipe(wmf.utils.df_to_remarkup)

| platform | editor_interface | abort | firstChange | init | loaded | ready | saveAttempt | saveFailure | saveIntent | saveSuccess
| ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- 
| desktop | visualeditor | 3.4% | 12.6% | 4.3% | 4.1% | 4.1% | 12.0% | 13.2% | 11.9% | 0.1%
| desktop | wikitext | 51.4% | 88.0% | 0.0% | 70.1% | 70.1% | 0.0% | 0.0% | 0.0% | 0.0%
| desktop | wikitext-2017 | 0.1% | 0.1% | 0.1% | 0.0% | 0.1% | 0.0% | 0.1% | 0.0% | 0.0%
| phone | visualeditor | 0.0% | 4.0% | 4.3% | 3.7% | 3.7% | 3.5% | 4.2% | 3.7% | 0.1%
| phone | wikitext | 1.5% | 3.6% | 5.5% | 4.9% | 4.9% | 3.5% | 3.3% | 3.2% | 0.1%

